In [25]:
!pip3 install tensorflow

The folder you are executing pip from can no longer be found.


In [78]:
!pip3 install Pillow

The folder you are executing pip from can no longer be found.


In [26]:
import os
import random
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import joblib

In [13]:
import os

face_data_dir = 'kaggleData/dataset1'

if os.path.exists(face_data_dir):
    print(f"Contents of '{face_data_dir}':")
    for item in os.listdir(face_data_dir):
        item_path = os.path.join(face_data_dir, item)
        print(f"- {item}")
else:
    print(f"Error: Directory '{face_data_dir}' not found.")

Contents of 'kaggleData/dataset1':
- .DS_Store
- test
- train
- validation


In [14]:
import os

face_data_dir = 'kaggleData/dataset2'

if os.path.exists(face_data_dir):
    print(f"Contents of '{face_data_dir}':")
    for item in os.listdir(face_data_dir):
        item_path = os.path.join(face_data_dir, item)
        print(f"- {item}")
else:
    print(f"Error: Directory '{face_data_dir}' not found.")

Contents of 'kaggleData/dataset2':
- .DS_Store
- test
- train
- validation


In [12]:
import os
import random
import shutil

main_dataset_dir = 'kaggleData'
dataset_name = 'dataset1'
dataset1_path = os.path.join(main_dataset_dir, dataset_name)

# Define the directories to sample
data_dirs = {'train': 'sampled_train',
             'test': 'sampled_test',
             'validation': 'sampled_val'}  # Adjust 'val' if your validation dir is named differently

sampling_ratio = 0.6  # Keep 20% of the original data

for original_subdir, sampled_subdir_name in data_dirs.items():
    original_dir = os.path.join(dataset1_path, original_subdir)
    sampled_dir = os.path.join(dataset1_path, sampled_subdir_name)
    os.makedirs(sampled_dir, exist_ok=True)

    if os.path.exists(original_dir) and os.path.isdir(original_dir):
        for class_name in os.listdir(original_dir):
            original_class_dir = os.path.join(original_dir, class_name)
            sampled_class_dir = os.path.join(sampled_dir, class_name)
            os.makedirs(sampled_class_dir, exist_ok=True)

            if os.path.isdir(original_class_dir):
                image_files = [f for f in os.listdir(original_class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
                num_to_sample = int(len(image_files) * sampling_ratio)
                sampled_files = random.sample(image_files, num_to_sample)

                for file_name in sampled_files:
                    original_path = os.path.join(original_class_dir, file_name)
                    sampled_path = os.path.join(sampled_class_dir, file_name)
                    shutil.copy2(original_path, sampled_path)  # Copy with metadata
        print(f"Sampling of '{original_subdir}' complete. Sampled data saved to: {sampled_dir}")
    else:
        print(f"Warning: Directory '{original_dir}' not found.")

print("Sampling process completed for train, test, and validation (if present).")

Sampling of 'train' complete. Sampled data saved to: kaggleData/dataset1/sampled_train
Sampling of 'test' complete. Sampled data saved to: kaggleData/dataset1/sampled_test
Sampling of 'validation' complete. Sampled data saved to: kaggleData/dataset1/sampled_val
Sampling process completed for train, test, and validation (if present).


In [33]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers.legacy import Adam  # Using legacy optimizer for M1/M2
from tensorflow.keras.preprocessing import image_dataset_from_directory

# --- Configuration ---
main_dataset_dir = 'kaggleData'
dataset_name = 'dataset1'
dataset1_path = os.path.join(main_dataset_dir, dataset_name)
image_height = 128
image_width = 128
batch_size = 32
epochs = 30  # Increased epochs
learning_rate = 0.0005  # Slightly reduced learning rate to start

num_classes = 2

# --- Define paths to your sampled directories ---
sampled_train_dir = os.path.join(dataset1_path, 'sampled_train')
sampled_val_dir = os.path.join(dataset1_path, 'sampled_val')
sampled_test_dir = os.path.join(dataset1_path, 'sampled_test')

# --- Load and Prepare Data ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    sampled_train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=True
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    sampled_val_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    sampled_test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=False
)

class_names = train_dataset.class_names
print("Class names:", class_names)
print("Number of classes:", num_classes, "\n")

# --- Tweaked Model Architecture ---
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),  # Added dropout for regularization
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Train the Model ---
print("Starting training of the tweaked model from scratch...")
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset
)

print("\nTraining finished!")

# --- Evaluate on the test set ---
print("\nEvaluating on the test set...")
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# --- Model Summary ---
model.summary()

Found 29809 files belonging to 2 classes.
Found 8389 files belonging to 2 classes.
Found 3859 files belonging to 3 classes.
Class names: ['fake', 'real']
Number of classes: 2 

Starting training of the tweaked model from scratch...
Epoch 1/30
932/932 [==============================] - 198s 212ms/step - loss: 0.6005 - accuracy: 0.7177 - val_loss: 0.5067 - val_accuracy: 0.7585
Epoch 2/30
932/932 [==============================] - 240s 258ms/step - loss: 0.2869 - accuracy: 0.8771 - val_loss: 0.6148 - val_accuracy: 0.7685
Epoch 3/30
932/932 [==============================] - 237s 254ms/step - loss: 0.1963 - accuracy: 0.9219 - val_loss: 0.3318 - val_accuracy: 0.8529
Epoch 4/30
932/932 [==============================] - 240s 258ms/step - loss: 0.1556 - accuracy: 0.9377 - val_loss: 0.2242 - val_accuracy: 0.9038
Epoch 5/30
932/932 [==============================] - 229s 245ms/step - loss: 0.1382 - accuracy: 0.9436 - val_loss: 0.4375 - val_accuracy: 0.8547
Epoch 6/30
932/932 [==================

KeyboardInterrupt: 

In [40]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers.legacy import Adam  # Using legacy optimizer for M1/M2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping

# --- Configuration ---
main_dataset_dir = 'kaggleData'
dataset_name = 'dataset1'
dataset1_path = os.path.join(main_dataset_dir, dataset_name)
image_height = 128
image_width = 128
batch_size = 32
epochs = 30
learning_rate = 0.0005
num_classes = 2
l2_lambda = 0.0001  # You can experiment with this value

# --- Define paths to your sampled directories ---
sampled_train_dir = os.path.join(dataset1_path, 'sampled_train')
sampled_val_dir = os.path.join(dataset1_path, 'sampled_val')
sampled_test_dir = os.path.join(dataset1_path, 'sampled_test')

# --- Load and Prepare Data ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    sampled_train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=True
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    sampled_val_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    sampled_test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=False
)

class_names = train_dataset.class_names
print("Class names:", class_names)
print("Number of classes:", num_classes, "\n")

# --- Tweaked Model Architecture with L2 Regularization ---
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3),
                  kernel_regularizer=regularizers.l2(l2_lambda)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_lambda))
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Early Stopping Callback ---
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# --- Train the Model with Early Stopping ---
print("Starting training with L2 regularization and early stopping...")
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)

print("\nTraining finished!")

# --- Evaluate on the test set ---
print("\nEvaluating on the test set...")
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# --- Model Summary ---
model.summary()

Found 29809 files belonging to 2 classes.
Found 8389 files belonging to 2 classes.
Found 3859 files belonging to 3 classes.
Class names: ['fake', 'real']
Number of classes: 2 

Starting training with L2 regularization and early stopping...
Epoch 1/30
932/932 [==============================] - 204s 219ms/step - loss: 0.5851 - accuracy: 0.7477 - val_loss: 0.5811 - val_accuracy: 0.7623
Epoch 2/30
932/932 [==============================] - 207s 222ms/step - loss: 0.3128 - accuracy: 0.8941 - val_loss: 0.4154 - val_accuracy: 0.8385
Epoch 3/30
932/932 [==============================] - 220s 236ms/step - loss: 0.2501 - accuracy: 0.9229 - val_loss: 0.3374 - val_accuracy: 0.8927
Epoch 4/30
932/932 [==============================] - 224s 241ms/step - loss: 0.2212 - accuracy: 0.9370 - val_loss: 0.3326 - val_accuracy: 0.8882
Epoch 5/30
932/932 [==============================] - 226s 243ms/step - loss: 0.2058 - accuracy: 0.9452 - val_loss: 0.3587 - val_accuracy: 0.8750
Epoch 6/30
932/932 [==========

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/sathvikanadipalli/Library/Python/3.8/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/Users/sathvikanadipalli/Library/Python/3.8/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/Users/sathvikanadipalli/Library/Python/3.8/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/Users/sathvikanadipalli/Library/Python/3.8/lib/python/site

Pivoting Prhect Ideas 

In [1]:
import tensorflow as tf
import os

# Define the main directory of your dataset
main_dataset_dir = 'Midjourney_Exp2'  # Replace with the actual path

# Define image size and batch size for MobileNetV3
image_height = 224
image_width = 224
batch_size = 32

# Function to load a dataset from a directory
def load_dataset(directory):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='binary',  # Assuming 'FAKE' and 'REAL' will be 0 and 1
        image_size=(image_height, image_width),
        batch_size=batch_size,
        shuffle=True  # Shuffle the training data
    )
    return dataset

# Load the training, validation, and test datasets
train_dir = os.path.join(main_dataset_dir, 'train')
valid_dir = os.path.join(main_dataset_dir, 'valid')
test_dir = os.path.join(main_dataset_dir, 'test')

sample_train = load_dataset(train_dir)
sample_valis = load_dataset(valid_dir)
sample_test = load_dataset(test_dir)

# Print some information about the loaded datasets
print("Sample Training Dataset:")
for image_batch, labels_batch in sample_train.take(1):
    print("Image batch shape:", image_batch.numpy().shape)
    print("Labels batch shape:", labels_batch.numpy().shape)

print("\nSample Validation Dataset:")
for image_batch, labels_batch in sample_valis.take(1):
    print("Image batch shape:", image_batch.numpy().shape)
    print("Labels batch shape:", labels_batch.numpy().shape)

print("\nSample Test Dataset:")
for image_batch, labels_batch in sample_test.take(1):
    print("Image batch shape:", image_batch.numpy().shape)
    print("Labels batch shape:", labels_batch.numpy().shape)

# Optional: Preprocessing function to normalize pixel values
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

sample_train = sample_train.map(preprocess)
sample_valis = sample_valis.map(preprocess)
sample_test = sample_test.map(preprocess)

print("\nDatasets with preprocessing applied:")
for image_batch, labels_batch in sample_train.take(1):
    print("Processed image batch dtype:", image_batch.dtype)
    break

Found 1001 files belonging to 2 classes.
Found 388 files belonging to 2 classes.
Found 229 files belonging to 2 classes.
Sample Training Dataset:
Image batch shape: (32, 224, 224, 3)
Labels batch shape: (32, 1)

Sample Validation Dataset:
Image batch shape: (32, 224, 224, 3)
Labels batch shape: (32, 1)

Sample Test Dataset:
Image batch shape: (32, 224, 224, 3)
Labels batch shape: (32, 1)

Datasets with preprocessing applied:
Processed image batch dtype: <dtype: 'float32'>


In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# Getting pretrained MobileNetV3 model

from tensorflow.keras.applications import MobileNetV3Small  # Or MobileNetV3Large
from tensorflow.keras import layers, models

# Choose the MobileNetV3 variant (Small is generally more efficient)
base_model = MobileNetV3Small(
    include_top=False,  # Don't include the classification head
    weights='imagenet', # Load weights pre-trained on ImageNet
    input_shape=(image_height, image_width, 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add a custom classification head
global_average_layer = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(1, activation='sigmoid')  # Binary classification (real/fake)

# Create the final model
model = models.Sequential([
    base_model,
    global_average_layer,
    layers.Dropout(0.5),
    prediction_layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary to see the architecture
model.summary()

4334752/4334752 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Function  (None, 7, 7, 576)         939120    
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 576)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 576)               0         
                                                                 
 dense (Dense)               (None, 1)                 577       
                                                                 
Total params: 939697 (3.58 MB)
Trainable params: 577 (2.25 KB)
Non-trainable params: 939120 (3.58 MB)
_______________________

In [5]:
epochs = 10  # You can adjust the number of epochs

history = model.fit(
    sample_train,
    epochs=epochs,
    validation_data=sample_valis
)

Epoch 1/10
32/32 [==============================] - 12s 315ms/step - loss: 0.8256 - accuracy: 0.5145 - val_loss: 0.6725 - val_accuracy: 0.6031
Epoch 2/10
32/32 [==============================] - 11s 302ms/step - loss: 0.7075 - accuracy: 0.6014 - val_loss: 0.6027 - val_accuracy: 0.6881
Epoch 3/10
32/32 [==============================] - 11s 305ms/step - loss: 0.6509 - accuracy: 0.6444 - val_loss: 0.5724 - val_accuracy: 0.7294
Epoch 4/10
32/32 [==============================] - 11s 304ms/step - loss: 0.5902 - accuracy: 0.6913 - val_loss: 0.5648 - val_accuracy: 0.7216
Epoch 5/10
32/32 [==============================] - 11s 300ms/step - loss: 0.5752 - accuracy: 0.7043 - val_loss: 0.5390 - val_accuracy: 0.7397
Epoch 6/10
32/32 [==============================] - 11s 300ms/step - loss: 0.5285 - accuracy: 0.7353 - val_loss: 0.5297 - val_accuracy: 0.7577
Epoch 7/10
32/32 [==============================] - 11s 301ms/step - loss: 0.5315 - accuracy: 0.7473 - val_loss: 0.5180 - val_accuracy: 0.7526